# Simulation of a single chain

In [1]:
import feasst as fst
print(fst.version())

def end_to_end_distance(mc):
    config = mc.system().configuration()
    pos = config.particle(0).site(0).position()
    return pos.distance(config.particle(0).site(config.particle(0).num_sites() - 1).position())

mc = fst.MonteCarlo()
mc.set(fst.MakeRandomMT19937(fst.args({"seed": "time"})))
mc.add(fst.Configuration(fst.MakeDomain(fst.args({"cubic_box_length": "20"})),
                         fst.args({"particle_type": fst.install_dir() + "/plugin/chain/forcefield/data.chain5"})))
mc.add(fst.MakePotential(fst.MakeLennardJones(),
                         fst.MakeVisitModelIntra(fst.args({"cutoff": "1"}))))
mc.set(fst.MakeThermoParams(fst.args({"beta": "1", "chemical_potential0": "1"})))
mc.set(fst.MakeMetropolis())
fst.SeekNumParticles(1).with_trial_add().run(mc)
mc.add(fst.MakeTrialPivot(fst.args({"weight": "1", "tunable_param": "20"})))
mc.add(fst.MakeTrialCrankshaft(fst.args({"weight": "1", "tunable_param": "20"})))
mc.add(fst.MakeTrialReptate(fst.args({"weight": "1", "max_length": "1"})))
mc.add(fst.MakeTrialGrowLinear(fst.MakeTrialComputeMove(),
                               fst.args({"weight": "1", "particle_type": "0", "num_steps": "3"})))
steps_per = str(int(1e4))
mc.add(fst.MakeLogAndMovie(fst.args({"steps_per": steps_per, "file_name": "chain"})))
mc.add(fst.MakeCheckEnergyAndTune(fst.args({"steps_per": steps_per, "tolerance": str(1e-10)})))
mc.add(fst.MakeCheckRigidBonds(fst.args({"steps_per": steps_per})))
print('Potential energy of initial configuration:', mc.criteria().current_energy())
rg_sq = fst.Accumulator()
print('end to end', end_to_end_distance(mc))
mc.attempt(int(1e3)) # equilibrate
for n in range(int(1e5)):
    mc.attempt()
    rg_sq.accumulate(end_to_end_distance(mc))

print(rg_sq.str())

v0.16.0-dirty-hwh/chaintutorial
Potential energy of initial configuration: -0.1900497542442386
end to end 4.0
average,stdev,block_stdev,n,moment0,moment1,
2.0392525217046433,0.58853031776533626,0,100000,203925.25217046433,450491.53185283651,
